In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import time
from Functions import *



In [20]:
service_url0 = 'https://ghoapi.azureedge.net/api/DIMENSION/COUNTRY/DimensionValues/'
response0 = requests.get(service_url0)

# make sure we got a valid response
print(response0)
if (response0.ok):
    # get the full data from the response
    data0j = response0.json()
    print(data0j.keys())
else:
    print("Response was not OK")

data0a = pd.DataFrame(data0j["value"])
data0a = data0a[data0a['Title'] != 'SPATIAL_SYNONYM']

remove_list = ['PRI', 'KNA', 'DMA', 'PSE', 'AND', 'SMR', 'MCO', 'LIE', 'COK', 
               'TUV', 'PLW', 'TKL', 'MHL', 'NIU', 'NRU', 'ME1', 'SDF']


<Response [200]>
dict_keys(['@odata.context', 'value'])


In [23]:
data0a[data0a.Code=='USA']

,Code,Title,ParentDimension,Dimension,ParentCode,ParentTitle
228,USA,United States of America,REGION,COUNTRY,AMR,Americas


In [8]:
do = pd.DataFrame()

In [12]:
type(do)

pandas.core.frame.DataFrame

In [5]:
data0a.head()

,Code,Title,ParentDimension,Dimension,ParentCode,ParentTitle
0,ABW,Aruba,REGION,COUNTRY,AMR,Americas
1,AFG,Afghanistan,REGION,COUNTRY,EMR,Eastern Mediterranean
2,AGO,Angola,REGION,COUNTRY,AFR,Africa
3,AIA,Anguilla,REGION,COUNTRY,AMR,Americas
4,ALB,Albania,REGION,COUNTRY,EUR,Europe


In [26]:

ind_code = 'WHOSIS_000001'
ind_text = 'life_expect'
data_raw = get_indicator(ind_code, ind_text)

data01 = data_raw[data_raw['Dim1'] == 'BTSX'][['country_code', 'year', ind_text]]
data01 = remove_duplicates(data01)
test_dump(data01)

Data for "life_expect" loaded, set 2196 rows 23 columns
(732, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 732 entries, 8 to 2195
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  732 non-null    object 
 1   year          732 non-null    int64  
 2   life_expect   732 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 22.9+ KB
None
    country_code  year  life_expect
140          BEL  2000     77.66169
141          BEL  2010     79.84602
142          BEL  2015     80.71184
143          BEL  2019     81.42305
country_code 183
year 4
life_expect 732


In [10]:
data01

,country_code,year,life_expect
8,AFG,2000,54.98949
9,AFG,2010,59.94055
10,AFG,2015,61.65429
11,AFG,2019,63.20990
20,AGO,2000,49.30265
...,...,...,...
2183,ZMB,2019,62.45290
2192,ZWE,2000,46.57054
2193,ZWE,2010,51.49335
2194,ZWE,2015,58.47603


In [18]:
data01 = data01.merge(data02, how='left', on=["country_code","year"])

In [19]:
data01

,country_code,year,life_expect,life_exp60,adult_mortality
0,AFG,2000,54.98949,13.92182,316.0496
1,AFG,2010,59.94055,15.12046,254.7747
2,AFG,2015,61.65429,15.58858,233.1531
3,AFG,2019,63.20990,15.21116,NaN
4,AGO,2000,49.30265,14.41537,383.5583
...,...,...,...,...,...
727,ZMB,2019,62.45290,16.53152,NaN
728,ZWE,2000,46.57054,14.41137,675.3416
729,ZWE,2010,51.49335,14.11814,500.4332
730,ZWE,2015,58.47603,15.10877,345.5254


In [13]:
pd.set_option('display.max_columns', None)

In [16]:
data_raw[data_raw.country_code=="AFG"]

,Id,IndicatorCode,SpatialDimType,country_code,TimeDimType,year,Dim1Type,Dim1,Dim2Type,Dim2,Dim3Type,Dim3,DataSourceDimType,DataSourceDim,Value,life_expect,Low,High,Comments,Date,TimeDimensionValue,TimeDimensionBegin,TimeDimensionEnd
0,24614313,WHOSIS_000001,COUNTRY,AFG,YEAR,2000,SEX,MLE,None,None,None,None,None,None,54.6,54.57449,None,None,None,2020-12-04T16:59:42.513+01:00,2000,2000-01-01T00:00:00+01:00,2000-12-31T00:00:00+01:00
1,24614317,WHOSIS_000001,COUNTRY,AFG,YEAR,2010,SEX,MLE,None,None,None,None,None,None,59.6,59.60036,None,None,None,2020-12-04T16:59:43.013+01:00,2010,2010-01-01T00:00:00+01:00,2010-12-31T00:00:00+01:00
2,24614321,WHOSIS_000001,COUNTRY,AFG,YEAR,2015,SEX,MLE,None,None,None,None,None,None,61.0,61.03658,None,None,None,2020-12-04T16:59:43.423+01:00,2015,2015-01-01T00:00:00+01:00,2015-12-31T00:00:00+01:00
3,24614325,WHOSIS_000001,COUNTRY,AFG,YEAR,2019,SEX,MLE,None,None,None,None,None,None,63.3,63.28709,None,None,None,2020-12-04T16:59:43.533+01:00,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00
4,24614329,WHOSIS_000001,COUNTRY,AFG,YEAR,2000,SEX,FMLE,None,None,None,None,None,None,55.4,55.41726,None,None,None,2020-12-04T16:59:43.61+01:00,2000,2000-01-01T00:00:00+01:00,2000-12-31T00:00:00+01:00
5,24614333,WHOSIS_000001,COUNTRY,AFG,YEAR,2010,SEX,FMLE,None,None,None,None,None,None,60.3,60.29720,None,None,None,2020-12-04T16:59:43.72+01:00,2010,2010-01-01T00:00:00+01:00,2010-12-31T00:00:00+01:00
6,24614337,WHOSIS_000001,COUNTRY,AFG,YEAR,2015,SEX,FMLE,None,None,None,None,None,None,62.3,62.34584,None,None,None,2020-12-04T16:59:43.847+01:00,2015,2015-01-01T00:00:00+01:00,2015-12-31T00:00:00+01:00
7,24614341,WHOSIS_000001,COUNTRY,AFG,YEAR,2019,SEX,FMLE,None,None,None,None,None,None,63.2,63.15551,None,None,None,2020-12-04T16:59:43.927+01:00,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00
8,24614345,WHOSIS_000001,COUNTRY,AFG,YEAR,2000,SEX,BTSX,None,None,None,None,None,None,55.0,54.98949,None,None,None,2020-12-04T16:59:44.053+01:00,2000,2000-01-01T00:00:00+01:00,2000-12-31T00:00:00+01:00
9,24614349,WHOSIS_000001,COUNTRY,AFG,YEAR,2010,SEX,BTSX,None,None,None,None,None,None,59.9,59.94055,None,None,None,2020-12-04T16:59:44.18+01:00,2010,2010-01-01T00:00:00+01:00,2010-12-31T00:00:00+01:00


In [18]:
data01[data01.country_code=="AFG"]

,country_code,year,life_expect
8,AFG,2000,54.98949
9,AFG,2010,59.94055
10,AFG,2015,61.65429
11,AFG,2019,63.20990


In [14]:
ind_code = 'WHOSIS_000015'
ind_text = 'life_exp60'
data_raw = get_indicator(ind_code, ind_text)

data01a = data_raw[data_raw['Dim1'] == 'BTSX'][['country_code', 'year', ind_text]]
data01a = remove_duplicates(data01a)
test_dump(data01a)

Data for "life_exp60" loaded, set 2196 rows 23 columns
(732, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 732 entries, 8 to 2195
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  732 non-null    object 
 1   year          732 non-null    int64  
 2   life_exp60    732 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 22.9+ KB
None
    country_code  year  life_exp60
140          BEL  2000    21.63166
141          BEL  2010    23.10626
142          BEL  2015    23.58500
143          BEL  2019    24.02222
country_code 183
year 4
life_exp60 732


In [31]:
ind_code = 'WHOSIS_000004'
ind_text = 'adult_mortality'
data_raw = get_indicator(ind_code, ind_text)

data02 = data_raw[data_raw['Dim1'] == 'BTSX'][['country_code', 'year', ind_text]]
data02 = remove_duplicates(data02)
test_dump(data02)

Data for "adult_mortality" loaded, set 9333 rows 23 columns
(3111, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3111 entries, 308 to 9689
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country_code     3111 non-null   object 
 1   year             3111 non-null   int64  
 2   adult_mortality  3111 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 97.2+ KB
None
    country_code  year  adult_mortality
869          BEL  2000         99.34914
872          BEL  2001         98.10686
875          BEL  2002         96.57720
878          BEL  2003         93.56242
881          BEL  2004         89.45644
884          BEL  2005         88.14542
887          BEL  2006         84.60459
890          BEL  2007         85.75175
893          BEL  2008         85.98980
896          BEL  2009         84.16703
899          BEL  2010         81.23653
902          BEL  2011         80.80521
905 

In [47]:
data = pd.read_csv(r'C:\Users\pabli\Desktop\Health-Database\data\columns.csv')

In [54]:
data

,Code,IndName
0,AIR_42,Ambient air pollution attributable death rate ...
1,AIR_51,Household air pollution attributable death rat...
2,anc1,Antenatal care coverage - at least one visit (...
3,BP_04,Raised blood pressure (SBP>=140 OR DBP>=90) (a...
4,BP_06,Mean systolic blood pressure (age-standardized...
...,...,...
75,MALARIA_ITN_COVERAGE,Population with access to an insecticide-treat...
76,Adult_curr_cig_smoking,Prevalence of current cigarette smoking among ...
77,MALARIA_EST_MORTALITY,Estimated malaria mortality rate (per 100 000 ...
78,vmsl,Measles immunization coverage among one-year-o...


In [48]:
all_ = pd.read_csv(r'C:\Users\pabli\Desktop\Health-Database\data\extraction_data.csv')

In [49]:
data.rename(columns={"IndCode": "Code"},inplace=True)

In [50]:
data.drop(columns = "Unnamed: 0", inplace = True)

In [51]:
all_.rename(columns={"Code": "country_code"},inplace=True)

In [52]:
all_

,Unnamed: 0.1,Unnamed: 0,country,country_code,year,Meningitis,Alzheimer's Disease and Other Dementias,Parkinson's Disease,Nutritional Deficiencies,Malaria,Drowning,Interpersonal Violence,Maternal Disorders,HIV/AIDS,Drug Use Disorders,Tuberculosis,Cardiovascular Diseases,Lower Respiratory Infections,Neonatal Disorders,Alcohol Use Disorders,Self-harm,Exposure to Forces of Nature,Diarrheal Diseases,Environmental Heat and Cold Exposure,Neoplasms,Conflict and Terrorism,Diabetes Mellitus,Chronic Kidney Disease,Poisonings,Protein-Energy Malnutrition,Road Injuries,Chronic Respiratory Diseases,Cirrhosis and Other Chronic Liver Diseases,Digestive Diseases,"Fire, Heat, and Hot Substances",Acute Hepatitis,population,per_capita-Meningitis,per_capita-Alzheimer's Disease and Other Dementias,per_capita-Parkinson's Disease,per_capita-Nutritional Deficiencies,per_capita-Malaria,per_capita-Drowning,per_capita-Interpersonal Violence,per_capita-Maternal Disorders,per_capita-HIV/AIDS,per_capita-Drug Use Disorders,per_capita-Tuberculosis,per_capita-Cardiovascular Diseases,per_capita-Lower Respiratory Infections,per_capita-Neonatal Disorders,per_capita-Alcohol Use Disorders,per_capita-Self-harm,per_capita-Exposure to Forces of Nature,per_capita-Diarrheal Diseases,per_capita-Environmental Heat and Cold Exposure,per_capita-Neoplasms,per_capita-Conflict and Terrorism,per_capita-Diabetes Mellitus,per_capita-Chronic Kidney Disease,per_capita-Poisonings,per_capita-Protein-Energy Malnutrition,per_capita-Road Injuries,per_capita-Chronic Respiratory Diseases,per_capita-Cirrhosis and Other Chronic Liver Diseases,per_capita-Digestive Diseases,"per_capita-Fire, Heat, and Hot Substances",per_capita-Acute Hepatitis,gdp,gdp/percap
0,0,0,Afghanistan,AFG,2000,3105.0,1291.0,416.0,3060.0,263.0,2048.0,2871.0,4271.0,97.0,186.0,5864.0,53919.0,32395.0,22920.0,99.0,1137.0,0.0,11323.0,52.0,13977.0,5429.0,2515.0,4182.0,499.0,3013.0,6581.0,7471.0,3249.0,6206.0,465.0,4334.0,22461348.0,0.000138,0.000057,0.000019,0.000136,0.000012,0.000091,0.000128,0.000190,0.000004,0.000008,0.000261,0.002401,0.001442,0.001020,0.000004,0.000051,0.000000e+00,0.000504,0.000002,0.000622,2.417041e-04,0.000112,0.000186,0.000022,0.000134,0.000293,0.000333,0.000145,0.000276,0.000021,0.000193,4.367000e+09,194.422881
1,1,1,Afghanistan,AFG,2001,3063.0,1300.0,424.0,2973.0,211.0,2044.0,2971.0,4385.0,104.0,197.0,5900.0,54642.0,31594.0,23276.0,101.0,1183.0,4.0,11446.0,380.0,14312.0,5689.0,2657.0,4251.0,502.0,2924.0,6720.0,7565.0,3267.0,6277.0,467.0,4363.0,22507460.0,0.000136,0.000058,0.000019,0.000132,0.000009,0.000091,0.000132,0.000195,0.000005,0.000009,0.000262,0.002428,0.001404,0.001034,0.000004,0.000053,1.777189e-07,0.000509,0.000017,0.000636,2.527606e-04,0.000118,0.000189,0.000022,0.000130,0.000299,0.000336,0.000145,0.000279,0.000021,0.000194,4.553000e+09,202.288486
2,2,2,Afghanistan,AFG,2002,2990.0,1311.0,429.0,2795.0,2883.0,1980.0,3277.0,4498.0,111.0,202.0,5660.0,54213.0,30346.0,23976.0,101.0,1190.0,1286.0,11393.0,53.0,14437.0,977.0,2745.0,4239.0,490.0,2746.0,6696.0,7455.0,3238.0,6246.0,455.0,4251.0,23600088.0,0.000127,0.000056,0.000018,0.000118,0.000122,0.000084,0.000139,0.000191,0.000005,0.000009,0.000240,0.002297,0.001286,0.001016,0.000004,0.000050,5.449132e-05,0.000483,0.000002,0.000612,4.139815e-05,0.000116,0.000180,0.000021,0.000116,0.000284,0.000316,0.000137,0.000265,0.000019,0.000180,5.146000e+09,218.050034
3,3,3,Afghanistan,AFG,2003,3300.0,1326.0,434.0,3039.0,2188.0,2282.0,3458.0,4702.0,118.0,216.0,5676.0,54274.0,32361.0,24628.0,104.0,1235.0,137.0,12348.0,55.0,14951.0,864.0,2865.0,4336.0,531.0,2984.0,7243.0,7471.0,3342.0,6443.0,499.0,4373.0,25005388.0,0.000132,0.000053,0.000017,0.000122,0.000088,0.000091,0.000138,0.000188,0.000005,0.000009,0.000227,0.002170,0.001294,0.000985,0.000004,0.000049,5.478819e-06,0.000494,0.000002,0.000598,3.455255e-05,0.000115,0.000173,0.000021,0.000119,0.000290,0.000299,0.000134,0.000258,0.000020,0.000175,6.167000e+09,246.626847
4,4,4,Afghanistan,AFG,2004,3350.0,1344

In [55]:
for i,e in enumerate(data.Code):     
    ind_code = e
    ind_text = data.IndName[i]
    data_raw = get_indicator(ind_code, ind_text)

    data01a = data_raw[['country_code', 'year', ind_text]]
    data01a = remove_duplicates(data01a)
    all_ = all_.merge(data01a, how='left',on = ['year','country_code'])

Data for "Ambient air pollution attributable death rate (per 100 000 population, age-standardized)" loaded, set 7341 rows 23 columns
Data for "Household air pollution attributable death rate (per 100 000 population)" loaded, set 7287 rows 23 columns
Data for "Antenatal care coverage - at least one visit (in the two or three years preceding the survey) (%" loaded, set 11874 rows 23 columns
Data for "Raised blood pressure (SBP>=140 OR DBP>=90) (age-standardized estimate)" loaded, set 9360 rows 23 columns
Data for "Mean systolic blood pressure (age-standardized estimate)" loaded, set 6240 rows 23 columns
Data for "Poverty headcount ratio at $1.25 a day (PPP) (% of population)" loaded, set 105 rows 23 columns
Data for "Human development index rank" loaded, set 369 rows 23 columns
Data for "Number of infant deaths" loaded, set 12285 rows 23 columns
Data for "Number of neonatal deaths" loaded, set 4095 rows 23 columns
Data for "Total density per 100 000 population: District/rural hospitals" 

Data for "Estimated malaria mortality rate (per 100 000 population)" loaded, set 2266 rows 23 columns
Data for "Measles immunization coverage among one-year-olds (%)" loaded, set 12598 rows 23 columns
Data for "Polio immunization coverage among one-year-olds (%)" loaded, set 12598 rows 23 columns


In [64]:
all_.head(20)

,Unnamed: 0.1,Unnamed: 0,country,country_code,year,Meningitis,Alzheimer's Disease and Other Dementias,Parkinson's Disease,Nutritional Deficiencies,Malaria,Drowning,Interpersonal Violence,Maternal Disorders,HIV/AIDS,Drug Use Disorders,Tuberculosis,Cardiovascular Diseases,Lower Respiratory Infections,Neonatal Disorders,Alcohol Use Disorders,Self-harm,Exposure to Forces of Nature,Diarrheal Diseases,Environmental Heat and Cold Exposure,Neoplasms,Conflict and Terrorism,Diabetes Mellitus,Chronic Kidney Disease,Poisonings,Protein-Energy Malnutrition,Road Injuries,Chronic Respiratory Diseases,Cirrhosis and Other Chronic Liver Diseases,Digestive Diseases,"Fire, Heat, and Hot Substances",Acute Hepatitis,population,per_capita-Meningitis,per_capita-Alzheimer's Disease and Other Dementias,per_capita-Parkinson's Disease,per_capita-Nutritional Deficiencies,per_capita-Malaria,per_capita-Drowning,per_capita-Interpersonal Violence,per_capita-Maternal Disorders,per_capita-HIV/AIDS,per_capita-Drug Use Disorders,per_capita-Tuberculosis,per_capita-Cardiovascular Diseases,per_capita-Lower Respiratory Infections,per_capita-Neonatal Disorders,per_capita-Alcohol Use Disorders,per_capita-Self-harm,per_capita-Exposure to Forces of Nature,per_capita-Diarrheal Diseases,per_capita-Environmental Heat and Cold Exposure,per_capita-Neoplasms,per_capita-Conflict and Terrorism,per_capita-Diabetes Mellitus,per_capita-Chronic Kidney Disease,per_capita-Poisonings,per_capita-Protein-Energy Malnutrition,per_capita-Road Injuries,per_capita-Chronic Respiratory Diseases,per_capita-Cirrhosis and Other Chronic Liver Diseases,per_capita-Digestive Diseases,"per_capita-Fire, Heat, and Hot Substances",per_capita-Acute Hepatitis,gdp,gdp/percap,"Ambient air pollution attributable death rate (per 100 000 population, age-standardized)",Household air pollution attributable death rate (per 100 000 population),Antenatal care coverage - at least one visit (in the two or three years preceding the survey) (%,Raised blood pressure (SBP>=140 OR DBP>=90) (age-standardized estimate),Mean systolic blood pressure (age-standardized estimate),Poverty headcount ratio at $1.25 a day (PPP) (% of population),Human development index rank,Number of infant deaths,Number of neonatal deaths,Total density per 100 000 population: District/rural hospitals,Total density per 100 000 population: Provincial hospitals,Total density per 100 000 population: Specialized hospitals,"Total dementia hospital admissions (per 100,000)",Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%),Estimate of current cigarette smoking prevalence (%),Estimate of current tobacco use prevalence (%),Measles-containing-vaccine second-dose (MCV2) immunization coverage by the nationally recommended age (%),Median availability of selected generic medicines (%) - Public,Deaths due to tuberculosis among HIV-negative people (per 100 000 population),Incidence of tuberculosis (per 100 000 population per year),Population aged 15-24 years with comprehensive correct knowledge of HIV/AIDS (%),Maternal mortality ratio (per 100 000 live births),Prevalence of HIV among adults aged 15 to 49 (%),Number of suspected meningitis cases reported,Number of deaths,"Prevalence of overweight among adults, BMI &GreaterEqual; 25 (age-standardized estimate) (%)","Prevalence of obesity among adults, BMI &GreaterEqual; 30 (age-standardized estimate) (%)",Mean BMI (kg/m²) (age-standardized estimate),"Prevalence of obesity among children and adolescents, BMI > +2 standard deviations above the median (crude estimate) (%)",Prevalence of insufficient physical activity among adults aged 18+ years (age-standardized estimate) (%),"Probability (%) of dying between age 30 and exact age 70 from any of cardiovascular disease, cancer, diabetes, or chronic respiratory disease",Polio immunization coverage among one-year-olds (%)_x,Affordability - percentage of GDP per capita required to purchase 2000 cigarettes of the most sold brand,Average - taxes 

In [57]:
pd.set_option('display.max_columns', None)

In [58]:
all_

,Unnamed: 0.1,Unnamed: 0,country,country_code,year,Meningitis,Alzheimer's Disease and Other Dementias,Parkinson's Disease,Nutritional Deficiencies,Malaria,Drowning,Interpersonal Violence,Maternal Disorders,HIV/AIDS,Drug Use Disorders,Tuberculosis,Cardiovascular Diseases,Lower Respiratory Infections,Neonatal Disorders,Alcohol Use Disorders,Self-harm,Exposure to Forces of Nature,Diarrheal Diseases,Environmental Heat and Cold Exposure,Neoplasms,Conflict and Terrorism,Diabetes Mellitus,Chronic Kidney Disease,Poisonings,Protein-Energy Malnutrition,Road Injuries,Chronic Respiratory Diseases,Cirrhosis and Other Chronic Liver Diseases,Digestive Diseases,"Fire, Heat, and Hot Substances",Acute Hepatitis,population,per_capita-Meningitis,per_capita-Alzheimer's Disease and Other Dementias,per_capita-Parkinson's Disease,per_capita-Nutritional Deficiencies,per_capita-Malaria,per_capita-Drowning,per_capita-Interpersonal Violence,per_capita-Maternal Disorders,per_capita-HIV/AIDS,per_capita-Drug Use Disorders,per_capita-Tuberculosis,per_capita-Cardiovascular Diseases,per_capita-Lower Respiratory Infections,per_capita-Neonatal Disorders,per_capita-Alcohol Use Disorders,per_capita-Self-harm,per_capita-Exposure to Forces of Nature,per_capita-Diarrheal Diseases,per_capita-Environmental Heat and Cold Exposure,per_capita-Neoplasms,per_capita-Conflict and Terrorism,per_capita-Diabetes Mellitus,per_capita-Chronic Kidney Disease,per_capita-Poisonings,per_capita-Protein-Energy Malnutrition,per_capita-Road Injuries,per_capita-Chronic Respiratory Diseases,per_capita-Cirrhosis and Other Chronic Liver Diseases,per_capita-Digestive Diseases,"per_capita-Fire, Heat, and Hot Substances",per_capita-Acute Hepatitis,gdp,gdp/percap,"Ambient air pollution attributable death rate (per 100 000 population, age-standardized)",Household air pollution attributable death rate (per 100 000 population),Antenatal care coverage - at least one visit (in the two or three years preceding the survey) (%,Raised blood pressure (SBP>=140 OR DBP>=90) (age-standardized estimate),Mean systolic blood pressure (age-standardized estimate),Poverty headcount ratio at $1.25 a day (PPP) (% of population),Human development index rank,Number of infant deaths,Number of neonatal deaths,Total density per 100 000 population: District/rural hospitals,Total density per 100 000 population: Provincial hospitals,Total density per 100 000 population: Specialized hospitals,"Total dementia hospital admissions (per 100,000)",Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%),Estimate of current cigarette smoking prevalence (%),Estimate of current tobacco use prevalence (%),Measles-containing-vaccine second-dose (MCV2) immunization coverage by the nationally recommended age (%),Median availability of selected generic medicines (%) - Public,Deaths due to tuberculosis among HIV-negative people (per 100 000 population),Incidence of tuberculosis (per 100 000 population per year),Population aged 15-24 years with comprehensive correct knowledge of HIV/AIDS (%),Maternal mortality ratio (per 100 000 live births),Prevalence of HIV among adults aged 15 to 49 (%),Number of suspected meningitis cases reported,Number of deaths,"Prevalence of overweight among adults, BMI &GreaterEqual; 25 (age-standardized estimate) (%)","Prevalence of obesity among adults, BMI &GreaterEqual; 30 (age-standardized estimate) (%)",Mean BMI (kg/m²) (age-standardized estimate),"Prevalence of obesity among children and adolescents, BMI > +2 standard deviations above the median (crude estimate) (%)",Prevalence of insufficient physical activity among adults aged 18+ years (age-standardized estimate) (%),"Probability (%) of dying between age 30 and exact age 70 from any of cardiovascular disease, cancer, diabetes, or chronic respiratory disease",Polio immunization coverage among one-year-olds (%)_x,Affordability - percentage of GDP per capita required to purchase 2000 cigarettes of the most sold brand,Average - taxes 

In [62]:
all_["Measles - number of reported cases"].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
3575    False
3576    False
3577    False
3578    False
3579    False
Name: Measles - number of reported cases, Length: 3580, dtype: bool